In [42]:
import collections
import random
import simpy

In [43]:
NUM_BOLETO = 120
TIEMPO_SIMULACION = 120

In [44]:
acciones_globales = {}
clientes = []

class Cliente:

  renegado: bool = False
  pelicula: str = 'NA'
  cantidad_boletos: int = 0
  acciones: dict = {}

  def __init__(self, genero):
    self.genero = genero
    pass

  def __str__(self):
    return  f'renegado {self.renegado}, pelicula {self.pelicula}, cantidad_boletos {self.cantidad_boletos}, acciones {self.acciones}'

In [48]:
def ventaBoletos(env, num_boletos, pelicula, teatro, cliente: Cliente):
  with teatro.contador.request() as turno:
    resultado = yield turno | teatro.sold_out[pelicula]

    if turno not in resultado:
      cliente.renegado = True
      cliente.acciones['renegado'] = env.now
      teatro.num_renegados[pelicula] += 1
      return

    if teatro.num_boletos[pelicula] < num_boletos:
      yield env.timeout(0.5)
      cliente.renegado = True
      cliente.acciones['sin_turno'] = env.now
      return
    teatro.num_boletos[pelicula] -= num_boletos
    if teatro.num_boletos[pelicula] < 2:
      acciones_globales['fin_boletos'+pelicula] = env.now
      teatro.sold_out[pelicula].succeed()
      teatro.tiempo_agotado[pelicula] = env.now
      teatro.num_boletos[pelicula] = 0
    yield env.timeout(1)


def llegadaClientes(env, teatro):
  while True:
    cliente = Cliente('NA')

    yield env.timeout(random.expovariate(1/0.5))
    pelicula = random.choice(teatro.peliculas)
    num_boletos = random.randint(1, 6)

    cliente.pelicula = pelicula
    cliente.cantidad_boletos = num_boletos
    cliente.acciones['llega'] = env.now

    if teatro.num_boletos[pelicula]:
      env.process(ventaBoletos(env, num_boletos, pelicula, teatro, cliente))
      clientes.append(cliente)


Teatro = collections.namedtuple('Teatro', 'contador, peliculas, num_boletos, sold_out, tiempo_agotado, num_renegados')

print('Teatro Carlos Crespi - UPS')
env = simpy.Environment()

contador = simpy.Resource(env,capacity=1)
peliculas = ['Conjuro 3', 'Rapidos y Furiosos 10', 'Pulp Fiction']
num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
sold_out = {pelicula: env.event() for pelicula in peliculas}
tiempo_agotado = {pelicula: None for pelicula in peliculas}
num_renegados = {pelicula: 0 for pelicula in peliculas}

teatro = Teatro(contador, peliculas, num_boletos, sold_out, tiempo_agotado, num_renegados)
env.process(llegadaClientes(env, teatro))
env.run(until=TIEMPO_SIMULACION)

# Analisis y resultados
for pelicula in peliculas:
  if teatro.sold_out[pelicula]:
    print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula]))
    print('Numero de personas que salieron de la fila/renegados %s' %teatro.num_renegados[pelicula])

Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 94.4 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 31
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 95.4 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 29
Pelicula: Pulp Fiction se agoto en el tiempo 88.9 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 34


In [46]:
acciones_globales

{'fin_boletosConjuro 3': 79.532115254801,
 'fin_boletosPulp Fiction': 93.532115254801,
 'fin_boletosRapidos y Furiosos 10': 101.532115254801}

In [47]:
for x in clientes:
  print(x)

renegado False, pelicula Rapidos y Furiosos 10, cantidad_boletos 3, acciones {'llega': 119.72845716071225, 'sin_turno': 77.532115254801, 'renegado': 101.532115254801}
renegado False, pelicula Pulp Fiction, cantidad_boletos 2, acciones {'llega': 119.72845716071225, 'sin_turno': 77.532115254801, 'renegado': 101.532115254801}
renegado False, pelicula Conjuro 3, cantidad_boletos 3, acciones {'llega': 119.72845716071225, 'sin_turno': 77.532115254801, 'renegado': 101.532115254801}
renegado False, pelicula Rapidos y Furiosos 10, cantidad_boletos 1, acciones {'llega': 119.72845716071225, 'sin_turno': 77.532115254801, 'renegado': 101.532115254801}
renegado False, pelicula Conjuro 3, cantidad_boletos 4, acciones {'llega': 119.72845716071225, 'sin_turno': 77.532115254801, 'renegado': 101.532115254801}
renegado False, pelicula Rapidos y Furiosos 10, cantidad_boletos 4, acciones {'llega': 119.72845716071225, 'sin_turno': 77.532115254801, 'renegado': 101.532115254801}
renegado False, pelicula Pulp F